In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

In [45]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import numpy as np
import pandas as pd

In [49]:
df = pd.DataFrame()

def extract_info_from_current_page():
    global df
    print("===> Jestem na stronie:", driver.current_url)

    wydzial_links = []
    wszystkie_wydzialy = driver.find_element(By.XPATH, '/html/body/main/div/div/section/div[2]')
    linki = wszystkie_wydzialy.find_elements(By.CSS_SELECTOR, 'div.elements-department a')

    for link in linki:
        href = link.get_attribute("href")
        if href and href.startswith("/"):
            href = base_url + href
        print("Znaleziony link:", href)
        wydzial_links.append(href)
    
    for href in wydzial_links:
        driver.get(href)
        time.sleep(0.1)
        wszytskie_kierunki = driver.find_element(By.ID, "main-content")

        links = wszytskie_kierunki.find_elements(By.CSS_SELECTOR, "li.student-view-content-list-item a")

        linki_kierunkow = [link.get_attribute("href") for link in links]

        for l in linki_kierunkow:
            metadane = {}
            driver.get(l)
            breadcrumb = driver.find_element(By.CSS_SELECTOR, "nav[aria-label='breadcrumb']")
            items = breadcrumb.find_elements(By.CSS_SELECTOR, "ol.breadcrumb li.breadcrumb-item")
            texts = [item.text.strip() for item in items[1:]]

            metadane['wydzial'] = texts[0]
            metadane['kierunek'] = texts[1]

            subtitle = driver.find_element(By.CSS_SELECTOR, ".section-subtitle")
            subtitle_list = [part.strip() for part in subtitle.text.split(",")]

            metadane['rok rozpoczecia'] = subtitle_list[0]
            metadane['typ 1'] = subtitle_list[1]
            metadane['typ 2'] = subtitle_list[2]

            container = driver.find_element(By.ID, "nav-tab-info-panel")
            first_paragraph = container.find_element(By.TAG_NAME, "p")
            opis_kierunku = first_paragraph.get_attribute("innerText").strip()

            metadane['opis kierunku'] = opis_kierunku
            # print(metadane)

            semestry = driver.find_element(By.ID, 'pills-tabContent')
            semestry = semestry.find_elements(By.CSS_SELECTOR, ":scope > div")[1:]
            
            for idx, semester in enumerate(semestry):
                metadane['semester'] = idx + 1

                specjalizacje = semester.find_elements(By.TAG_NAME, 'article')

                for specka in specjalizacje:
                    try:
                        nazwa_specki = specka.find_element(By.TAG_NAME, "h2")
                    except:
                        metadane['specka'] = None
                    else:
                        nazwa_specki = nazwa_specki.find_element(By.TAG_NAME, 'button')
                        nazwa_specki = nazwa_specki.find_elements(By.TAG_NAME, "span")[1]
                        metadane['specka'] = nazwa_specki.get_attribute("textContent").strip()

                    # print(metadane)
                    przedmioty = []
                    tablica = specka.find_element(By.TAG_NAME, 'table')

                    tablice = tablica.find_elements(By.TAG_NAME, 'tbody') # do zwyklego i obierakow

                    for tbody in tablice:
                        przedmioty.extend(go_through_tbody_element(tbody))
                    # print(przedmioty)

                    for p in przedmioty:
                        row = metadane.copy()
                        row['Przedmiot'] = p
                        df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)
                    
                    print(df.tail(1))
            print('aaa')


def go_through_tbody_element(element):
    przedmioty = []
    wiersze = element.find_elements(By.CSS_SELECTOR, ":scope > tr")

    for wiersz in wiersze:
        przedmiot = wiersz.find_element(By.TAG_NAME, "td")
        try:
            przedmiot = przedmiot.find_element(By.TAG_NAME, "button")
            nazwa_przedmiotu = przedmiot.get_attribute("textContent").strip()
        except:
            try:
                przedmiot = przedmiot.find_element(By.TAG_NAME, "a")
                nazwa_przedmiotu = przedmiot.get_attribute("textContent").strip()
            except:
                nazwa_przedmiotu = driver.execute_script("""let element = arguments[0];
                                                            let text = "";
                                                            for (let node of element.childNodes) {
                                                                if (node.nodeType === Node.TEXT_NODE) {
                                                                    text += node.textContent;
                                                                }
                                                            }
                                                            return text.trim();
                                                        """, przedmiot)

        if not (nazwa_przedmiotu.startswith("Język obcy") or nazwa_przedmiotu.startswith("Zasady wyboru") or nazwa_przedmiotu.startswith("Przedmiot")):
            przedmioty.append(nazwa_przedmiotu)
    return przedmioty
    


# Start
driver = webdriver.Chrome()
driver.get("https://sylabusy.agh.edu.pl/")

# 1. Wykonaj akcje na pierwszej stronie
extract_info_from_current_page()


# 2. Pobierz hrefy do dalszych stron
rozpoczecie_ul = driver.find_element(By.XPATH, "//div[contains(., 'Rozpoczęcie studiów')]/following-sibling::ul[1]")
link_elements = rozpoczecie_ul.find_elements(By.TAG_NAME, "a")
hrefs = [el.get_attribute("href") for el in link_elements]

# 3. Przejdź przez pozostałe linki
for href in hrefs:
    driver.get(href)
    time.sleep(0.1)  # ewentualnie WebDriverWait
    extract_info_from_current_page()
    # break

driver.quit()

===> Jestem na stronie: https://sylabusy.agh.edu.pl/
Znaleziony link: https://sylabusy.agh.edu.pl/pl/1/2/21/0/0/2
Znaleziony link: https://sylabusy.agh.edu.pl/pl/1/2/21/0/0/3
Znaleziony link: https://sylabusy.agh.edu.pl/pl/1/2/21/0/0/16
Znaleziony link: https://sylabusy.agh.edu.pl/pl/1/2/21/0/0/4
Znaleziony link: https://sylabusy.agh.edu.pl/pl/1/2/21/0/0/5
Znaleziony link: https://sylabusy.agh.edu.pl/pl/1/2/21/0/0/6
Znaleziony link: https://sylabusy.agh.edu.pl/pl/1/2/21/0/0/7
Znaleziony link: https://sylabusy.agh.edu.pl/pl/1/2/21/0/0/8
Znaleziony link: https://sylabusy.agh.edu.pl/pl/1/2/21/0/0/9
Znaleziony link: https://sylabusy.agh.edu.pl/pl/1/2/21/0/0/10
Znaleziony link: https://sylabusy.agh.edu.pl/pl/1/2/21/0/0/11
Znaleziony link: https://sylabusy.agh.edu.pl/pl/1/2/21/0/0/12
Znaleziony link: https://sylabusy.agh.edu.pl/pl/1/2/21/0/0/13
Znaleziony link: https://sylabusy.agh.edu.pl/pl/1/2/21/0/0/17
Znaleziony link: https://sylabusy.agh.edu.pl/pl/1/2/21/0/0/14
Znaleziony link: https://

TypeError: 'NoneType' object is not iterable